In [8]:

import re
import requests

import pdfplumber
import pandas as pd
from collections import namedtuple

# 尝试网上的几种提取pdf信息的方法， 先下载PDF


def download_file(url):
    local_filename = url.split('/')[-1]

    with requests.get(url) as r:
        with open(local_filename, 'wb') as f:
            f.write(r.content)

    return local_filename


ap_url = 'https://www.tabs3.com/support/sample/apreports.pdf'
ap = download_file(ap_url)  # download file apreports.pdf

# 1.pdfplumber
with pdfplumber.open(ap) as pdf:
    page = pdf.pages[15]
    pdf_text = page.extract_text()

In [9]:
print(pdf_text)  # 字句间隔有问题

AccountsPayableReportPack
invoicesenteredandcomparingthetotaltothechecksum,youcanensurethatalltransactions
havebeenenteredandassignedtothepropervendor.
ChangedEntriesand AnyentriesthatareeditedordeletedwillbelistedundertheChangedEntriesorDeletedEntries
DeletedEntries sections.Forchangedentries,theoriginalentrywillbeshownunderthe"Old"heading,andthe
changeswillbereflectedunderthe"New"heading.Whenaninvoiceormanualcheckisdeleted,
theInvoiceAmountisautomaticallychangedtozeroandtheStatusfieldischangedto"D".
TotalTransactionCount Totalnumberoftransactionsonthelistincludingchangesanddeletions.Eachchangedtransaction
(i.e.,botholdandnew)countsasonetransaction.
1Notshownonsamplereport.
-16-


In [25]:
# 2.pdfminer
from pdfminer.high_level import extract_pages, extract_text
text = extract_text("table.pdf")
print(text)

评级时间：2023 年 6 月 19 日

优势

跟踪评级报告

本次评级使用的评级方法、模型：

名称

版本

煤炭企业信用评级方法

V4.0.202208

煤炭企业主体信用评级模型（打分表）

V4.0.202208

注：上述评级方法和评级模型均已在联合资信官网公开披露

本次评级模型打分表及结果：

评价内容 评价结果 风险因素 评价要素 评价结果

经营
风险

财务
风险

A

F1

经营环境

自身
竞争力

宏观和区域
风险

行业风险

基础素质

企业管理

经营分析

资产质量

现金流

盈利能力

现金流量

资本结构

偿债能力

指示评级

个体调整因素

个体信用等级

外部支持调整因素

评级结果

2

5

1

2

1

1

2

1

2

1

aaa

--

aaa

--

AAA

注：经营风险由低至高划分为 A、B、C、D、E、F 共 6 个等级，各级因子评价划分
为 6 档，1 档最好，6 档最差；财务风险由低至高划分为 F1~F7 共 7 个等级，各级
因子评价划分为 7 档，1 档最好，7 档最差；财务指标为近三年加权平均值；通过矩
阵分析模型得到指示评级结果

分析师：余瑞娟 王 皓

邮箱：lianhe@lhratings.com

电话：010-85679696

传真：010-85679228

地址：北京市朝阳区建国门外大街 2 号

中国人保财险大厦 17 层（100022）

网址：www.lhratings.com

1. 煤炭资源及业务规模优势显著。截至 2022 年底，

公司煤炭资源总量 908.72 亿吨，可采储量为 180.55

亿吨，煤炭产能 3.24 亿吨/年。同时，公司具备年

产甲醇 531 万吨、醋酸 100 万吨、尿素 52 万吨、

焦炭 563 万吨的生产能力。

2. 整体经营业绩水平大幅提升，经营活动现金净流

入规模明显扩大。2022 年，公司实现营业总收入

8347.15 亿元，同比增长 10.72%；实现利润总额

423.59 亿元，同比增长 81.39%。同期，公司经营

活动现金净流入 988.89 亿元，同比增长 81.49%。

3. 收购中泰证券后，公司资产规模明显扩大。截至

2022 年底，公司合并资产总额 9511.2

In [23]:
page = extract_pages("Online_ticket.pdf")
for i in page:
    print(i)
    break

<LTPage(1) 0.000,0.000,595.276,836.221 rotate=0>


In [18]:

# pattern match tool
pattern = re.compile(r"[a-zA-Z]+,{1}\s{1}")
match = pattern.findall(text)
match[0:5]

['Technology, ', 'Lincoln, ', 'PracticeMaster, ', 'Pay, ', 'Technology, ']

In [24]:

# 3.PyPDF2 preferred extraction tool
import PyPDF2

fileobj = open("Online_ticket.pdf", 'rb')
pdfReader = PyPDF2.PdfFileReader(fileobj)
pages = pdfReader.numPages


for i in range(pages):
    pageObj = pdfReader.getPage(i)
    # Printing Page Number
    print("Page No: ", i)
    # Extracting text from page
    # And splitting it into chunks of lines
    text = pageObj.extractText().split("  ")
    for i in range(len(text)):
        print(text[i], end="\n\n")

Page No:  0
Online-Ticket
ICE Fahrkarte
Gültigkeit: Hinfahrt ab 12.12.2022, Rückfahrt ab 12.12.2022
Gilt nur für eingetragene Züge und Tage/Zeiten (Zugbindung).
NV=Nahverkehrszüge vor/nach Fernverkehrszügen.
Gültigkeitstag maßgebend für das Alter. Gilt nur für Reisende unter 27 Jahren bei
Reiseantritt (Alle Reisenden).
SuperSparpreis Young (Hin- und Rückfahrt)
Klasse: 2
Erw: 1 (15-26 J.)
Hinfahrt: München

Köln , mit ICE, (SSPY0612)
Rückfahrt: Köln

München , mit ICE, (SSPY0612)
Über: H: M-Hbf 8:48 ICE724 R: K-Hbf 17:55 ICE613
Storno ausgeschlossen.
Zahlungspositionen und Preis
Positionen Preis MwSt (D) 19% MwSt (D) 7%
ICE Fahrkarte 1 45,80€ 45,80€ 3,00€
Summe 45,80€ 45,80€ 3,00€
giropay-Zahlung
Betrag 45,80€ Transaktions-Nr

SUdFobGIhGf4pwbvckDp
Datum 25.11.2022
Der oben genannte Betrag wurde von Ihrem giropay-Konto eingezogen. Die Buchung
Ihres Online-Tickets erfolgte am 25.11.2022 17:12 Uhr. DB Fernverkehr AG, Europa-
Allee 78 - 84, 60486 Frankfurt am Main / DB Regio AG, Europa-Alle

In [25]:
# extract table from pdf

import tabula
dfs = tabula.read_pdf("table.pdf", pages='all', multiple_tables=True)
tables = {}
for i, df in enumerate(dfs):
    table_name = f"Table {i+1}"
    first_line = df.iloc[0, :]
    # print(first_line.values)
    tables[table_name] = df  # 这个方法不好：提取的表格会混合周围的文字信息
print(tables.keys())
# print(tables['Table 1'])
print(tables['Table 2'])

dict_keys(['Table 1', 'Table 2'])
               评级时间:2023年 6月 19日   优势                      Unnamed: 0
0                            NaN  1.0        煤炭资源及业务规模优势显著。截至 2022年底,
1                本次评级使用的评级方法、模型:  NaN  公司煤炭资源总量 908.72亿吨,可采储量为 180.55
2                          名称 版本  NaN       亿吨,煤炭产能 3.24亿吨/年。同时,公司具备年
3         煤炭企业信用评级方法 V4.0.202208  NaN                             NaN
4                            NaN  NaN     产甲醇 531万吨、醋酸 100万吨、尿素 52万吨、
5  煤炭企业主体信用评级模型(打分表) V4.0.202208  NaN                             NaN
6                            NaN  NaN                  焦炭 563万吨的生产能力。
7     注:上述评级方法和评级模型均已在联合资信官网公开披露  NaN                             NaN
8                            NaN  2.0           整体经营业绩水平大幅提升,经营活动现金净流


In [27]:
import fitz  # PyMUPDF
import PIL.Image  # pillow
import io

# tool for extracting image, saved as jpeg

pdf = fitz.open("Online_ticket.pdf")  # extract images from pdf
counter = 1
for i in range(len(pdf)):
    page = pdf[i]
    images = page.get_images()
    for image in images:
        base_img = pdf.extract_image(image[0])
        image_data = base_img["image"]
        img = PIL.Image.open(io.BytesIO(image_data))
        extension = base_img["ext"]
        img.save(open(f"image{counter}.{extension}", "wb"))
        counter += 1